In [48]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

In [49]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [50]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [51]:
# Remove the unwanted columns 
df = df.drop(['RowNumber', 'CustomerId', 'Surname'],axis=1)

In [52]:
# Label Encoder for categorical variables
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])

In [53]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [54]:
# OneHot Encode 'Geography'
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder(sparse_output=False)
geo_encoder = onehot_encoder_geo.fit_transform(df[['Geography']])

In [55]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [56]:
geo_encoded_df = pd.DataFrame(
    geo_encoder,
    columns=onehot_encoder_geo.get_feature_names_out(['Geography'])
)

In [57]:
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [58]:
## Combine onehot encoder columns with the original data
df = pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [59]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [60]:
# Split the Data into features and target
X = df.drop('EstimatedSalary',axis=1)
y = df['EstimatedSalary']

In [61]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,1,0.0,1.0,0.0


In [62]:
y

0       101348.88
1       112542.58
2       113931.57
3        93826.63
4        79084.10
          ...    
9995     96270.64
9996    101699.77
9997     42085.58
9998     92888.52
9999     38190.78
Name: EstimatedSalary, Length: 10000, dtype: float64

In [63]:
# Split the data into train,test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [64]:
# Scale these features 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [65]:
### Save the Encoder and Scaler into pickel file 

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
    
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)
    
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

### ANN Regression Problem Statement

In [66]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [67]:
## Build the ANN Model 
model  = Sequential(
    [
        Dense(64,activation = 'relu',input_shape=(X_train.shape[1],)), #Hidden Layer 1 connected with input layer,(we have to define input shape on 1st Hidden Layer)
        Dense(32,activation='relu'), #HL2
        Dense(1,activation='sigmoid') ## Output Layer
        
        
    ]
)

In [68]:
## Compile the model 
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [69]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [70]:
import datetime
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

## Stepup the Tensorboard
log_dir = 'regressionlogs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir,histogram_freq=1)

In [71]:
## Setup Early Stopping 
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [72]:
# Train the model 
history = model.fit(
    X_train,y_train,
    validation_data = (X_test,y_test),
    epochs=100,
    callbacks = [early_stopping_callback,tensorflow_callback] 
)

Epoch 1/100


250/250 [==============================] - 2s 3ms/step - loss: 100430.3203 - mae: 100430.3203 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 100430.2812 - mae: 100430.2812 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 100430.2969 - mae: 100430.2969 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 100430.3047 - mae: 100430.3047 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 100430.2969 - mae: 100430.2969 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 100430.2734 - mae: 100430.2734 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 100430.2969 - mae

In [73]:
%load_ext tensorboard

In [76]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 26696), started 0:00:38 ago. (Use '!kill 26696' to kill it.)

In [78]:
## Evaluate Model on the test data 

test_loss,test_mae = model.evaluate(X_test,y_test)

print(f'Test MAE : {test_mae}')

63/63 [==============================] - 0s 1ms/step - loss: 98725.0625 - mae: 98725.0625
Test MAE : 98725.0625


In [79]:
model.save('regression_model.h5')

c:\Users\Dell\anaconda3\envs\churn_env\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
